# DeepMIR

## 1. Load tool packages

In [1]:
from Metrics import Metrics
import wget
from Predict import predict

## 2. Load the data

In [2]:
wget.download('https://github.com/LinTan-CSU/DeepMIR/releases/download/OrganicSolventsDataset/Quaternary.npy', '../data/')
wget.download('https://github.com/LinTan-CSU/DeepMIR/releases/download/OrganicSolventsDataset/Quaternary_labels.csv', '../data/')

'../data//Quaternary_labels.csv'

## 3. Component Prediction

In [3]:
lib = ['1,2-dichloroethane', '1-butanol', 'acetonitrile', 'cyclohexane', 'dichloromethane', 'diethylene_glycol_dimethyl_ether', 'ethanol', 'hexane', 'isopropyl_alcohol', 'methanol', 'toluene', 'trichloromethane']
predict('../model/DeepMIR.pth', '../data/Quaternary.npy', lib)

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,diethylene_glycol_dimethyl_ether,0.997112,cyclohexane,0.995129,trichloromethane,0.973697,"1,2-dichloroethane",0.961720,methanol,0.000427,...,ethanol,2.048722e-08,isopropyl_alcohol,2.769045e-09,1-butanol,7.475618e-10,toluene,5.405035e-10,acetonitrile,1.609274e-10
1,diethylene_glycol_dimethyl_ether,0.998766,dichloromethane,0.994836,cyclohexane,0.809556,1-butanol,0.644998,hexane,0.358927,...,methanol,3.900311e-04,toluene,5.545476e-05,isopropyl_alcohol,4.321016e-05,"1,2-dichloroethane",1.013915e-05,acetonitrile,3.093833e-09
2,isopropyl_alcohol,0.999742,acetonitrile,0.976795,cyclohexane,0.975125,toluene,0.925490,hexane,0.241967,...,diethylene_glycol_dimethyl_ether,7.588649e-04,ethanol,4.899718e-04,1-butanol,3.229742e-07,methanol,1.189224e-09,"1,2-dichloroethane",5.633548e-10
3,acetonitrile,0.999983,diethylene_glycol_dimethyl_ether,0.997898,"1,2-dichloroethane",0.985959,cyclohexane,0.956041,hexane,0.005558,...,dichloromethane,3.735709e-06,ethanol,4.991081e-07,1-butanol,4.988178e-07,isopropyl_alcohol,8.154867e-09,toluene,4.289394e-09
4,diethylene_glycol_dimethyl_ether,0.996662,cyclohexane,0.992149,trichloromethane,0.984025,"1,2-dichloroethane",0.982622,methanol,0.000514,...,ethanol,2.029772e-08,isopropyl_alcohol,6.512747e-09,toluene,1.264057e-09,1-butanol,6.234342e-10,acetonitrile,2.340167e-10
5,"1,2-dichloroethane",0.999655,cyclohexane,0.978302,dichloromethane,0.969184,ethanol,0.961579,trichloromethane,0.034267,...,1-butanol,5.734857e-06,isopropyl_alcohol,2.746953e-07,acetonitrile,1.985467e-07,diethylene_glycol_dimethyl_ether,1.830769e-07,methanol,6.887760e-09
6,diethylene_glycol_dimethyl_ether,0.998259,trichloromethane,0.987152,"1,2-dichloroethane",0.966796,cyclohexane,0.051482,hexane,0.043262,...,1-butanol,9.523667e-08,isopropyl_alcohol,6.339534e-08,ethanol,3.264719e-08,acetonitrile,3.016786e-09,toluene,1.886456e-09
7,isopropyl_alcohol,0.999922,acetonitrile,0.994106,dichloromethane,0.894718,cyclohexane,0.868571,hexane,0.231956,...,"1,2-dichloroethane",9.557473e-05,toluene,4.953618e-06,ethanol,4.451213e-06,1-butanol,6.413697e-09,methanol,1.788826e-09
8,diethylene_glycol_dimethyl_ether,0.995279,cyclohexane,0.993068,trichloromethane,0.954561,methanol,0.328838,dichloromethane,0.000129,...,toluene,4.305035e-07,"1,2-dichloroethane",1.531434e-08,isopropyl_alcohol,9.259478e-10,acetonitrile,7.564879e-10,1-butanol,1.846196e-10
9,"1,2-dichloroethane",0.986049,cyclohexane,0.885970,toluene,0.805659,diethylene_glycol_dimethyl_ether,0.604515,methanol,0.000598,...,ethanol,2.766575e-05,acetonitrile,6.721791e-09,trichloromethane,1.161820e-09,isopropyl_alcohol,3.559505e-10,1-butanol,2.541496e-11


## 4. Metrics

In [5]:
TPR, TNR, ACC = Metrics('../res/DeepMIR.csv', '../data/Quaternary_labels.csv')
print("TPR:{:.4f}".format(TPR), "TNR:{:.4f}".format(TNR), "ACC:{:.4f}".format(ACC))

TPR:0.9500 TNR:1.0000 ACC:0.9833
